<a href="https://colab.research.google.com/github/ilkersigirci/ML-with-Colab/blob/master/text_similarity_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import time
from google.colab import drive
from pprint import pprint
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#cleaning text
import re
import nltk
from string import punctuation
from nltk.stem import SnowballStemmer

def pad_str(s):
    return ' '+s+' '
def pad_number(pattern):
    matched_string = pattern.group(0)
    return pad_str(matched_string)
def pad_pattern(pattern):
    matched_string = pattern.group(0)
    return pad_str(matched_string)

def clean(text):
    
    if pd.isnull(text):
        return ''
    
    # Empty question   
    if type(text) != str or text=='':
        return ''
    
    # Clean the text, with the option to stem words.
    # stops = set(stopwords.words("english"))
    
    ###########################################################################################
    ################################# SPECIFIC TO JOTFORM #####################################
    ###########################################################################################
    text = re.sub('<[^<]+?>', ' ',                  text, flags=re.IGNORECASE)  # remove html
    text = re.sub('/(\r\n)+|\r+|\n+|\t+/i', '' ,    text, flags=re.IGNORECASE)  # remove \r \n
    text = re.sub("nbsp", "",                       text, flags=re.IGNORECASE)  # remove nbsp
    text = ''.join([c for c in text if c not in punctuation]).lower()           # remove punc
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",     text, flags=re.IGNORECASE)  # remove tags
    text = re.sub("http\S*", "",                    text, flags=re.IGNORECASE)  # remove links
    text = re.sub("www\S*", "",                     text, flags=re.IGNORECASE)  # remove links
    text = re.sub("\S*@\S*\s?", "",                 text, flags=re.IGNORECASE)  # remove email
    text = re.sub("jo[a-z]*form", "jotform",        text, flags=re.IGNORECASE)  # fix1 jotform
    text = re.sub("jot[a-z]+", "jotform",           text, flags=re.IGNORECASE)  # fix2 jotform
    text = re.sub("jotform", "form",                text, flags=re.IGNORECASE)  # jotform to form
    text = re.sub("colour", "color",                text, flags=re.IGNORECASE)  # fix colour
    text = re.sub("authorizenet", "payment",        text, flags=re.IGNORECASE)
    text = re.sub("calender", "calendar",           text, flags=re.IGNORECASE)
    
    text = re.sub("autosuspended", "auto suspend",  text, flags=re.IGNORECASE)
    text = re.sub("autoresponses", "auto response", text, flags=re.IGNORECASE)
    text = re.sub("wordpresscom", "word press",     text, flags=re.IGNORECASE)
    text = re.sub("myforms", "my form",             text, flags=re.IGNORECASE)
    text = re.sub("onedrive", "cloud",              text, flags=re.IGNORECASE)
    text = re.sub("gmailcom", "email",              text, flags=re.IGNORECASE)
    text = re.sub("jo[a-z]*form", "form",           text, flags=re.IGNORECASE)
    text = re.sub("unrequire", "not require",       text, flags=re.IGNORECASE)
    text = re.sub("prepopulate[a-z]*", "populate",  text, flags=re.IGNORECASE)
    text = re.sub("sumbissions", "submission",      text, flags=re.IGNORECASE)
    text = re.sub("submissons", "submission",       text, flags=re.IGNORECASE)
    text = re.sub("mozilla\S*", "browser",          text, flags=re.IGNORECASE)
    text = re.sub("chrome\S*", "browser",           text, flags=re.IGNORECASE)
    
    
    ###########################################################################################
    ################################# SPECIFIC TO JOTFORM #####################################
    ###########################################################################################
    
    #Fix Negative
    text = re.sub("n't", " not ",                   text, flags=re.IGNORECASE)
    text = re.sub("isnt", "is not",                 text, flags=re.IGNORECASE)
    text = re.sub("arent", "are not",               text, flags=re.IGNORECASE)
    text = re.sub("dont", "do not",                 text, flags=re.IGNORECASE)
    text = re.sub("doesnt", "does not",             text, flags=re.IGNORECASE)
    text = re.sub("didnt", "did not",               text, flags=re.IGNORECASE)
    text = re.sub("shouldnt", "should not",         text, flags=re.IGNORECASE)
    text = re.sub("hasnt", "has not",               text, flags=re.IGNORECASE)
    text = re.sub("havent", "have not",             text, flags=re.IGNORECASE)
    text = re.sub("wont", "will not",               text, flags=re.IGNORECASE)
    text = re.sub("cant", "can not",                text, flags=re.IGNORECASE)
    text = re.sub("couldnt", "could not",           text, flags=re.IGNORECASE)
    
    text = re.sub("\'s", " ",                       text, flags=re.IGNORECASE)
    text = re.sub("whats", "what is",               text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ",                 text, flags=re.IGNORECASE)    
    text = re.sub("i'm", "i am",                    text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ",                  text, flags=re.IGNORECASE)
    text = re.sub("\'d", " would ",                 text, flags=re.IGNORECASE)
    text = re.sub("\'ll", " will ",                 text, flags=re.IGNORECASE)
    text = re.sub("e\.g\.", " eg ",                 text, flags=re.IGNORECASE)
    text = re.sub("e-mail", " email ",              text, flags=re.IGNORECASE)
    text = re.sub('\$', " dollar ",                 text, flags=re.IGNORECASE)
    text = re.sub('\%', " percent ",                text, flags=re.IGNORECASE)
    text = re.sub('\&', " and ",                    text, flags=re.IGNORECASE)
    text = re.sub("android","operating system",     text, flags=re.IGNORECASE)
    text = re.sub("ios", "operating system",        text, flags=re.IGNORECASE)
    text = re.sub('[0-9]+\.[0-9]+', "flt_number",   text, flags=re.IGNORECASE)  # replace the float numbers with flt_number    
    text = re.sub(" +"," ",                         text, flags=re.IGNORECASE)  # remove multiple space
    text = re.sub('[^\x00-\x7F]+', "",              text)                       # remove non-ascii characters
    
    ############################### LEMMATIZE ####################################
    
    from nltk import word_tokenize, pos_tag
    from nltk.stem.wordnet import WordNetLemmatizer
    #nltk.download('wordnet')
    #nltk.download('punkt')
    #nltk.download('averaged_perceptron_tagger')
    # n -> noun / v -> verb / a -> adjective / r -> adverb
    tokens = word_tokenize(text)
    convert_tag = lambda e: ('a' if e[0].lower() == 'j' else e[0].lower()) if e[0].lower() in ['n', 'r', 'v'] else 'n'    
    lemmatizer = WordNetLemmatizer()    
    text  = ' '.join([ lemmatizer.lemmatize(token, convert_tag(tag)) for token,tag in pos_tag(tokens) ])
    
    ############################### STOP WORDS ####################################
    #nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    text =  " ".join(word for word in text.split() if word not in stop_words)

    return text

In [0]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
nltk.download('punkt')

def initial_clean(text):
    """
    Function to clean text of websites, email addresess and any punctuation
    We also lower case the text
    """
    text = text.lower() # lower case the text
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = re.sub("[^a-zA-Z ]", "", text)    
    text = ''.join([c for c in text if c not in punctuation]).lower()           # remove punc
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",     text, flags=re.IGNORECASE)  # remove tags
    text = nltk.word_tokenize(text)
    return text

stop_words = stopwords.words('english')
def remove_stop_words(text):
    """
    Function that removes all stopwords from text
    """
    return [word for word in text if word not in stop_words]

#stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words, so plural and singular are treated the same
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    """
    This function applies all the functions above into one
    """
    #return stem_words(remove_stop_words(initial_clean(text)))
    return remove_stop_words(initial_clean(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!mkdir mydata

In [0]:
########################### ORIGINAL CSV #########################################

!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/original.csv  mydata/

pd.set_option('display.max_colwidth', -1)
dfPath = 'mydata/original.csv'
original = pd.read_csv(dfPath, sep='|',index_col=0)

#original = original.dropna(axis = 0)
#original["id"] = original["id"].apply(str)

#original_questions = original['question']
#original_questions = np.array(original_questions)

In [0]:
########################### PREPROCESSED CSV ####################################

!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/preprocessed.csv  mydata/

pd.set_option('display.max_colwidth', -1)
dfPath = 'mydata/preprocessed.csv'
df = pd.read_csv(dfPath, sep='|',index_col=0)

questions = df['all']
questions = np.array(questions)
original_all = df['question'] + "  / " + df["details"]

In [0]:
# split sentences
splitted = []
for sentence in questions:
    if type(sentence) == str:
        splitted.append(sentence.lower().split())

splitted = np.array(splitted)

In [0]:
########################################## TF-IDF ############################################## 
import nltk, string, numpy
nltk.download('punkt')
nltk.download('wordnet')

stemmer = nltk.stem.porter.PorterStemmer()
def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def StemNormalize(text):
    return StemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

###### TF-IDF HARD WAY (ram yetmiyor) ######## 
"""
from sklearn.feature_extraction.text import CountVectorizer
LemVectorizer = CountVectorizer(tokenizer=LemNormalize, stop_words='english')
LemVectorizer.fit_transform(documents)
print (LemVectorizer.vocabulary_)

tf_matrix = LemVectorizer.transform(documents).toarray() # out of ram error
print(tf_matrix)

# Inverse Document Freq (idf)
# terim ne kadar az dökumanda tekrar ediyor ise IDF değeri o kadar büyük çıkar.
from sklearn.feature_extraction.text import TfidfTransformer
    tfidfTran = TfidfTransformer(norm="l2") # euclidian distance
    tfidfTran.fit(tf_matrix)
    print tfidfTran.idf_
    
tfidf_matrix = tfidfTran.transform(tf_matrix)
print tfidf_matrix.toarray()
"""
###### TF-IDF EASY WAY ########
# TfidfVectorizer combines the work of CountVectorizer and TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer() # (tokenizer=LemNormalize, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# TF-IDF matrix for each document 4 -> (the number of rows of the matrix) 
# Tf-idf terms                    11-> (the number of columns from the matrix)
print (tfidf_matrix.shape)
print(tfidf_matrix)

In [0]:
### Cosine similarity with tf-idf
"""
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()
cos_similarity(documents)
"""
#Better way
#Cosine Similarity between the first document with all documents in the set
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_matrix[0], tfidf_matrix)

In [0]:
######################################### FuzzyWuzzy ############################################## 
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

# ratio            -> 51 -> compares the entire string similarity, in order
# partial_ratio    -> 52 -> compares partial string similarity.
# token_sort_ratio -> 46 -> ignores word order.
# token_set_ratio  -> 52 -> ignores duplicated words

print(fuzz.partial_token_set_ratio(documents[0], documents[1])) # 100
print(fuzz.partial_token_set_ratio(documents[0], documents[2])) # 100

In [0]:
######################################### Word2Vec ############################################## 

from gensim.models import Word2Vec

# size      -> determines the dimensions of the word vectors
# window    -> number of forward and backward words used in training
# min_count -> number of minimum times a word needs to appear to be included in the training
# sg        -> 0 = CBOW / 1 = skip gram

w2v_model      = Word2Vec(splitted, size = 100, window = 5, min_count = 3) # 0.62 min
w2v_model_sg2  = Word2Vec(splitted, size = 100, window = 5, min_count = 3,sg = 2) # 1.89 min
w2v_model_norm = Word2Vec(splitted, size = 100, window = 5, min_count = 3) # 0.60 min
w2v_model_norm.init_sims(replace=True) #normalize word2vec vectors

In [0]:
w2v_model.save('mydata/w2v_model_size100_window5_min3.model')
w2v_model_sg2.save('mydata/w2v_model_sg2_size100_window5_min3.model')
w2v_model_norm.save('mydata/w2v_model_norm_size100_window5_min3.model')
!cp mydata/w2v_model_size100_window5_min3.model drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/
!cp mydata/w2v_model_sg2_size100_window5_min3.model drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/
!cp mydata/w2v_model_norm_size100_window5_min3.model drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
from gensim.models import Word2Vec

#Save model
#w2v_model.save('mydata/w2v_model_size100_window5_min3.model')
#!cp mydata/w2v_model_size100_window5_min3.model drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/

#Load model
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/w2v_model_size100_window5_min3.model mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/w2v_model_sg2_size100_window5_min3.model mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/w2v_model_norm_size100_window5_min3.model mydata/
w2v_model      = Word2Vec.load('mydata/w2v_model_size100_window5_min3.model')
w2v_model_sg2  = Word2Vec.load('mydata/w2v_model_sg2_size100_window5_min3.model')
w2v_model_norm = Word2Vec.load('mydata/w2v_model_norm_size100_window5_min3.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
######################################### Glove ############################################## 
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip -P mydata/
!unzip mydata/glove.840B.300d.zip -d mydata/
!touch mydata/glove_word2vec.txt

from gensim.scripts.glove2word2vec import glove2word2vec
_ = glove2word2vec(r'mydata/glove.840B.300d.txt', r"mydata/glove_word2vec.txt")
##########################################################
from gensim.models import KeyedVectors

pre_model = KeyedVectors.load_word2vec_format(r"mydata/glove_word2vec.txt", binary=False)
model_glove = gensim.models.Word2Vec(size=100, min_count=5)
model_glove.build_vocab(splitted)
total_examples = model_glove.corpus_count
model_glove.build_vocab([list(pre_model.vocab.keys())], update=True)
model_glove.intersect_word2vec_format(r"mydata/glove_word2vec.txt", binary=False, lockf=1.0)
model_glove.train(tokenized, total_examples=total_examples, epochs=model_glove.iter)
##########################################################
print(model_glove.most_similar(positive=['blue'], topn=10))
print(w2v_model.most_similar(positive=['blue'], topn=10))

In [0]:
###################################### WMD Distance ###########################################

import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
#stop_words = stopwords.words('english')

def my_wmd(q1, q2,model_type):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    #q1 = [w for w in q1 if w not in stop_words]
    #q2 = [w for w in q2 if w not in stop_words]
    return model_type.wv.wmdistance(q1, q2)

#all_except_index = np.delete(a, ind, axis=0) 

t1 = time.time()

text = "how to change font in google forms"
my_question = clean(text)

wmd_distance = []
for question in questions:
    wmd_distance.append(my_wmd(my_question,question,w2v_model))
wmd_distance = np.array(wmd_distance)

t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min") # 12

"""
print("wmd normalize starting..")
wmd_distance_norm = []
for question in questions:
    wmd_distance_norm.append(my_wmd(my_question,question,w2v_model_norm))
wmd_distance_norm = np.array(wmd_distance_norm)
"""

In [0]:
# Get the indices of the four min elements -> returns unsorted
min_indexes = np.argpartition(wmd_distance, 10)[:10]
min_indexes = min_indexes[np.argsort(wmd_distance[min_indexes])] #sort it
#min_indexes_norm = np.argpartition(wmd_distance_norm, 4)[:4]
#min_indexes_norm = min_indexes_norm[np.argsort(wmd_distance_norm[min_indexes_norm])] #sort it

print("question             \n {}".format(text))
print("similar questions       \n {}".format(df['question'][min_indexes] + " " + df['details'][min_indexes]))
#print("similar questions norm  \n {}".format(questions[min_indexes_norm]))

In [0]:
def combine_wmd(my_question,similar,model_type,number): # questionlar string formda olacak
    t1 = time.time()
    
    wmd_distance = []
    for question in similar['all']:
        distance = model_type.wv.wmdistance(my_question.split(), question.split())        
        wmd_distance.append(distance)
        
    wmd_distance = np.array(wmd_distance)
    min_indexes = np.argpartition(wmd_distance, number)[:number]
    min_indexes = min_indexes[np.argsort(wmd_distance[min_indexes])]
    t2 = time.time()
    print("Calculation took ",t2-t1, "seconds")
    
    return min_indexes

In [0]:
###################################### WMD Similarity ###########################################

from gensim.similarities import WmdSimilarity
instance = WmdSimilarity(splitted, w2v_model, num_best=10)

t1 = time.time()

my_question = "How do i retrieve the forms I created prior to upgrading?"
my_question = clean(my_question)

sims = instance[my_question]  # A query is simply a "look-up" in the similarity class.

t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 28 min

# Print the query and the retrieved documents, together with their similarities.
print ('Query:')
print (my_question)
for i in range(10):
    print
    print ('sim = %.4f' % sims[i][1])
    print (questions[sims[i][0]])

In [0]:
######################################### Sent2Vec ##############################################

from gensim.models import Sent2Vec

s2v_model      = Sent2Vec(splitted, size = 100, window = 5, min_count = 1)
s2v_model_norm = Sent2Vec(splitted, size = 100, window = 5, min_count = 1)
s2v_model_norm.init_sims(replace=True) #normalize word2vec vectors

In [0]:
nltk.download('punkt')
from nltk import word_tokenize

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(w2v_model[w])
        except:
            continue
    M = np.array(M)
    print(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
#Ram patliyor
from tqdm import tqdm_notebook

question_vectors = np.zeros((df.shape[0], 10))

for i, q in enumerate(tqdm_notebook(questions)):
    question_vectors[i, :] = sent2vec(q)

In [0]:
cosine_distance = [cosine(x, y) for q1_vec in np.nan_to_num(question_vectors)]


In [0]:
######################################### Doc2Vec ##############################################
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

tagged_questions = [TaggedDocument(doc, [i]) for i, doc in enumerate(splitted)]
#d2v_model = Doc2Vec(tagged_questions, size = 100, window = 10, min_count = 1)


# dm=1         -> distributed Memory(PV-DM)  -> preserves word order -> same as CBOW
# dm=0         -> distributed BOW   (PV-DBOW)-> same as skipgram
# dbow_words=1 -> add skip-gram word-training
# hs           -> hierarchical-softmax, better but slower / default is disable
d2v_model = Doc2Vec(dm = 0, min_count=2, window=10, size=100, sample=1e-4, negative=5,iter=10,workers=4)
d2v_model.build_vocab(tagged_questions)

for epoch in range(20):
    d2v_model.train(tagged_questions,total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)
    print("Epoch #{} is complete.".format(epoch+1))
    
# Store the model to mmap-able files
#d2v_model.save('d2v_dm0_epoch10.model')
# Load the model
#d2v_model = Doc2Vec.load('d2v_dm0_epoch10.model')

In [9]:
#Save model
#d2v_model.save('mydata/d2v_dm0_epoch10.model')
#!cp mydata/d2v_dm0_epoch10.model drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/
#!cp mydata/d2v_dm0_epoch10.model.docvecs.vectors_docs.npy drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/

#Load model
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/d2v_dm0_epoch10.model mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/d2v_dm0_epoch10.model.docvecs.vectors_docs.npy mydata/
d2v_model = Doc2Vec.load('mydata/d2v_dm0_epoch10.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
text = "Please cancel my subscription and refund the amount."


def doc2vec_similar(text):
    my_question = preprocess(text) # splitted formda
    my_question = " ".join(my_question)
    
    new_vector = d2v_model.infer_vector(my_question.split(), epochs=50, alpha=0.25)
    #gives you top k document tags and their cosine similarity
    similars = d2v_model.docvecs.most_similar([new_vector],topn=7000)

    similar_index = [index[0] for index in similars]
    d2v_similar = df.loc[similar_index]
    d2v_similar.reset_index(drop=True,inplace=True)

    # Combine d2v with WMD
    min_indexes_d2v = combine_wmd(my_question,d2v_similar,w2v_model,10)

    print("question             \n {}".format(text))
    print("similar questions    \n {}".format(d2v_similar['question'][min_indexes_d2v] + " / " + d2v_similar['details'][min_indexes_d2v]))

#score = d2v_model.n_similarity(questions[1],questions[2])

In [0]:
######################################### LDA ##############################################

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(7)

import nltk
nltk.download('wordnet')

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
data_questions = df[["all"]]
data_questions['index'] = data_questions.index
questions = data_questions

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in text.split():
        token = clean(token)
        if len(token) > 2:
            #result.append(lemmatize_stemming(token))
            result.append(token)
        
    return result

In [0]:
processed_questions = splitted
np.random.seed(7)

In [0]:
################ Bag of words on the dataset ############################
dictionary = gensim.corpora.Dictionary(processed_questions)

#less than 15 documents (absolute number) or
#more than 0.5 documents (fraction of total corpus size, not absolute number).
#keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=70000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_questions]

In [0]:
####################### TF-IDF ############################

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [0]:
########### Running LDA using Bag of Words #################
np.random.seed(7) # for getting same results each time.
t1 = time.time()

lda_model  = gensim.models.LdaMulticore(bow_corpus, num_topics=100, id2word=dictionary, passes=5, workers=4) # 0.45
lda_model_2 = gensim.models.LdaMulticore(bow_corpus, num_topics=150, id2word=dictionary, passes=5, workers=4) # 0.45

t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 10 min

In [0]:
#pprint(lda_model.print_topics())
for idx, topic in lda_model_2.print_topics(-1):
    print('Topic: {} Words: {}'.format(idx, topic))

In [16]:
from gensim.models.ldamulticore import LdaMulticore

#Save model
#lda_model.save("mydata/lda_model_topic150_pass5.model")
#!cp mydata/lda_model_topic150_pass5.model                 drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/
#!cp mydata/lda_model_topic150_pass5.model.expElogbeta.npy drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/
#!cp mydata/lda_model_topic150_pass5.model.id2word         drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/
#!cp mydata/lda_model_topic150_pass5.model.state           drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/

#Load model

!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_topic150_pass5.model                   mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_topic150_pass5.model.expElogbeta.npy   mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_topic150_pass5.model.id2word           mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_topic150_pass5.model.state             mydata/

!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_2_topic150_pass5.model                 mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_2_topic150_pass5.model.expElogbeta.npy mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_2_topic150_pass5.model.id2word         mydata/
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/models/lda/lda_model_2_topic150_pass5.model.state           mydata/

lda_model   = LdaMulticore.load('mydata/lda_model_topic150_pass5.model')
lda_model_2 = LdaMulticore.load('mydata/lda_model_2_topic150_pass5.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
########### Running LDA using TF-IDF #################
t1 = time.time()

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=100, id2word=dictionary, passes=5, workers=4)
lda_model_tfidf.save("mydata/lda_model_tfidf_topic100_pass5.model")
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 8 min

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [0]:
# Visualize lda model

# Fairly big, non-overlapping bubbles instead of being clustered in one quadrant.  -> GOOD MODEL
# Too many topics typically have many overlaps, small bubbles clustered in one region.

#!pip install pyLDAvis
import pyLDAvis.gensim
#import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

display_lda =  pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
#pyLDAvis.display(display_lda)
display_lda

In [0]:
############################ LDA MALLET ###############################
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip -P mydata/
!unzip mydata/mallet-2.0.8.zip -d mydata/
mallet_path = 'mydata/mallet-2.0.8/bin/mallet' # update this path

t1 = time.time()
lda_model = gensim.models.wrappers.LdaMallet(mallet_path, bow_corpus, num_topics=100, id2word=dictionary, workers=3) #, passes=5
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took min

In [0]:
############################ TESTING MODEL COHERENCE ###########################3
from gensim.models import CoherenceModel

# topic100        = 0.45
# topic100_mallet = 0.51
# topic50_mallet  = 0.57

# Compute Coherence Score
#coherence_model_lda = CoherenceModel(model=lda_model, texts=splitted, dictionary=dictionary, coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda) # 0.51

def compute_coherence_values(dictionary, corpus, texts, limit, start=50, step=50):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary,workers=4)
        print("model {} is finished".format(num_topics))
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        print("coherence {} is finished".format(num_topics))
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

t1 = time.time()
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow_corpus,  
                                                        texts=splitted, start=20, limit=60, step=20)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 23 min


# Show graph
limit=60; start=20; step=20;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [0]:
############################ DISTANCE TRIALS #################################

from scipy.spatial.distance import jensenshannon
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full, cossim, softcossim
from sklearn.metrics.pairwise import cosine_similarity

def distance_computations(dataframe,all_lda_corpus,que_lda,distance_metric,number,select):    

    t1 = time.time()
    distances = []
    for each_que_lda in all_lda_corpus:
        distances.append( distance_metric(que_lda, each_que_lda) )

    distances = np.array(distances)
    # Get the indices of the number max-min elements -> returns unsorted
    if(select == "min")  :
        indexes = np.argpartition(distances, number)[:number]
        indexes = indexes[np.argsort(distances[indexes])] #sort it -> ascending
    elif(select == "max"): 
        indexes = np.argpartition(distances, -number)[-number:] 
        indexes = indexes[np.argsort(distances[indexes])][::-1] #sort it -> descending

    similar = dataframe.loc[indexes]
    similar.reset_index(drop=True,inplace=True)
    t2 = time.time()
    print("Calculation took ",t2-t1, "seconds")
    
    my_print(similar,indexes,distance_metric,"original")
    
    return similar


def my_print(similar,indexes,distance_metric,select):
    print("####################################################################################################")
    
    if(select == "original"):
        print("similar questions -> {}    \n {}".format( distance_metric, original_all[indexes][:10] ))
    elif(select == "similar"):   
        print("similar questions    \n {}".format(similar['question'][indexes] + " / " + similar['details'][indexes]))
    
    print("####################################################################################################")

In [0]:
#questions_final = df['all']
#questions_final = np.array(questions_final)

In [18]:
t1 = time.time()
all_lda = [lda_model[x] for x in bow_corpus]
all_lda_2 = [lda_model_2[x] for x in bow_corpus]
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 5 min

Calculation took  6.963758293787638 min


In [48]:

#text = "Is it possible to transfer a form to another jotform account?"
#text = "Please cancel my subscription and refund the amount."
#text = "I am unable to free up the upload space even after deleting submissions."
text = "Can i see the time user spend to fill the form"
#text = "Unable to view photos upon submission of form"
#text = "One column not wide enough and other is too wide"
#text = "How to create conditional thank you page?"
#text = "Allow multiple date selection and to not lock the selected dates"
que_question = preprocess(text) # splitted formda
que_question = preprocess(text) # splitted formda
print(que_question)
#que_question = splitted[100]
#que_bow = [dictionary.doc2bow(que_question)]  # dogrusu bu gibi ama hata veriyor
que_bow   = dictionary.doc2bow(que_question)
que_lda   = lda_model[que_bow]
que_lda_2 = lda_model_2[que_bow]

['see', 'time', 'user', 'spend', 'fill', 'form']


In [0]:
doc2vec_similar(text)

In [0]:
# hellinger

print("my_question             \n {}".format(text))
similar_hellinger   = distance_computations(df, all_lda,   que_lda,   hellinger, 7000, "min")
similar_2_hellinger = distance_computations(df, all_lda_2, que_lda_2, hellinger, 7000, "min")

# Combine hellinger with WMD
que_question = " ".join(que_question)

print("question             \n {}".format(text))
min_indexes_wmd = combine_wmd(que_question,similar_hellinger,w2v_model,10)
my_print(similar_hellinger,min_indexes_wmd,hellinger,"similar")
min_indexes_2_wmd = combine_wmd(que_question,similar_2_hellinger,w2v_model,10)
my_print(similar_2_hellinger,min_indexes_2_wmd,hellinger,"similar")

In [0]:
# cosine

print("my_question             \n {}".format(text))
similar_cos   = distance_computations(df, all_lda,   que_lda,   cossim, 7000,"max")
similar_2_cos = distance_computations(df, all_lda_2, que_lda_2, cossim, 7000,"max")

# Combine hellinger with WMD
que_question = " ".join(que_question)

print("question             \n {}".format(text))
min_indexes_wmd = combine_wmd(que_question,similar_cos,w2v_model,10)
my_print(similar_cos,min_indexes_wmd,cossim,"similar")
min_indexes_2_wmd = combine_wmd(que_question,similar_2_cos,w2v_model,10)
my_print(similar_2_cos,min_indexes_2_wmd,cossim,"similar")

In [0]:
# kullback_leibler #not symmetric       ->  hata veriyor bazen

kullback_leibler_distance = []
for each_que_lda in all_lda:
    kullback_leibler_distance.append( kullback_leibler(que_lda, each_que_lda) )
    
kullback_leibler_distance = np.array(kullback_leibler_distance)

# Get the indices of the four min elements -> returns unsorted
min_indexes = np.argpartition(kullback_leibler_distance, 4)[:4]
min_indexes = min_indexes[np.argsort(kullback_leibler_distance[min_indexes])] #sort it

print("my_question             \n {}".format(que_question))
print("similar questions       \n {}".format(questions_final[min_indexes]))

In [0]:
# jensenshannon
from scipy.spatial.distance import jensenshannon
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full

shanjen_distance = []
for each_que_lda in all_lda:
    shanjen_distance.append( jensenshannon(que_lda, each_que_lda) )
    
shanjen_distance = np.array(shanjen_distance)

In [0]:
##################################################################################################################################################################################################################
###################################################################################### END #######################################################################################################################
##################################################################################################################################################################################################################

In [0]:
#################### jensenshannon manual ##############################

In [0]:
que_question = processed_questions[80]
new_bow = dictionary.doc2bow(que_question)

t1 = time.time()
new_doc_distribution = np.array([tup[1] for tup in lda_model.get_document_topics(bow=new_bow)])
t2 = time.time()
#print("Calculation took ",(t2-t1)/60, "min")  # Took 4 min


Calculation took  3.689130147298177e-05 min


In [0]:
# we need to use nested list comprehension here
t1 = time.time()
doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda_model[bow_corpus]])
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 2 min

Calculation took  2.2370501160621643 min


In [0]:
doc_topic_dist.shape

(218519,)

In [0]:
def jensen_shannon(query, matrix):
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix
    m = 0.5*(p + q)
    return np.sqrt(0.5*(entropy(p,m) + entropy(q,m)))

In [0]:
def get_most_similar_documents(query,matrix,k=10):
    """
    This function implements the Jensen-Shannon distance above
    and retruns the top k indices of the smallest jensen shannon distances
    """
    sims = jensen_shannon(query,matrix) # list of jensen shannon distances
    return sims.argsort()[:k] # the top k positional index of the smallest Jensen Shannon distances

In [0]:
# this is surprisingly fast
t1 = time.time()
most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 5 min

In [0]:
most_sim_ids

In [0]:
# Get the indices of the four min elements -> returns unsorted
min_indexes = get_most_similar_documents(new_doc_distribution,doc_topic_dist)
min_indexes = np.array(min_indexes)

print("my_question             \n {}".format(que_question))
print("similar questions       \n {}".format(questions_final[min_indexes]))